In [18]:
import argparse
import os
import shutil
import time
import yaml
import sys
import gdown
import numpy as np
import torch
import torch.distributed as dist
import torch.multiprocessing as mp
import torch.nn as nn
import torch.nn.functional as F
from monai.config import KeysCollection
from monai.metrics import Cumulative, CumulativeAverage
from monai.networks.nets import milmodel, resnet, MILModel

from sklearn.metrics import cohen_kappa_score
from torch.cuda.amp import GradScaler, autocast
from torch.utils.data.dataloader import default_collate
from torchvision.models.resnet import ResNet50_Weights
import shutil
from pathlib import Path
from torch.utils.data.distributed import DistributedSampler
from torch.utils.tensorboard import SummaryWriter
from monai.utils import set_determinism
import matplotlib.pyplot as plt
import wandb
import math
import logging
from pathlib import Path


from src.model.MIL import MILModel_3D
from src.model.csPCa_model import csPCa_Model
from src.data.data_loader import get_dataloader
from src.utils import save_cspca_checkpoint, get_metrics, setup_logging, save_pirads_checkpoint
from src.train import train_cspca, train_pirads
import SimpleITK as sitk 

import nrrd

from tqdm import tqdm
import pandas as pd
from picai_prep.preprocessing import PreprocessingSettings, Sample
import multiprocessing
import sys
from src.preprocessing.register_and_crop import register_files
from src.preprocessing.prostate_mask import get_segmask
from src.preprocessing.histogram_match import histmatch
from src.preprocessing.generate_heatmap import get_heatmap
import logging
from pathlib import Path
from src.utils import setup_logging
from src.utils import validate_steps
import argparse
import yaml 
from src.data.data_loader import data_transform, list_data_collate
from monai.data import Dataset, load_decathlon_datalist, ITKReader, NumpyReader, PersistentDataset

In [ ]:

args = argparse.Namespace(

    margin = 0.2,
    t2_dir = '/sc-scratch/sc-scratch-cc06-ag-ki-radiologie/prostate_foundation/WSAttention-Prostate/datatemp/t2',
    dwi_dir = '/sc-scratch/sc-scratch-cc06-ag-ki-radiologie/prostate_foundation/WSAttention-Prostate/datatemp/dwi',
    adc_dir = '/sc-scratch/sc-scratch-cc06-ag-ki-radiologie/prostate_foundation/WSAttention-Prostate/datatemp/adc',
    output_dir = '/sc-scratch/sc-scratch-cc06-ag-ki-radiologie/prostate_foundation/WSAttention-Prostate/datatemp/processed',
    project_dir = '/sc-scratch/sc-scratch-cc06-ag-ki-radiologie/prostate_foundation/WSAttention-Prostate'
)

args = register_files(args)
args = get_segmask(args)
args = histmatch(args)
args = get_heatmap(args)


  0%|          | 0/1 [00:02<?, ?it/s]
You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  0%|          | 0/1 [00:07<?, ?it/s]


In [6]:
vars(args)

{'margin': 0.2,
 't2_dir': '/sc-scratch/sc-scratch-cc06-ag-ki-radiologie/prostate_foundation/WSAttention-Prostate/datatemp/processed/t2_histmatched',
 'dwi_dir': '/sc-scratch/sc-scratch-cc06-ag-ki-radiologie/prostate_foundation/WSAttention-Prostate/datatemp/processed/DWI_histmatched',
 'adc_dir': '/sc-scratch/sc-scratch-cc06-ag-ki-radiologie/prostate_foundation/WSAttention-Prostate/datatemp/processed/ADC_histmatched',
 'output_dir': '/sc-scratch/sc-scratch-cc06-ag-ki-radiologie/prostate_foundation/WSAttention-Prostate/datatemp/processed',
 'project_dir': '/sc-scratch/sc-scratch-cc06-ag-ki-radiologie/prostate_foundation/WSAttention-Prostate',
 'seg_dir': '/sc-scratch/sc-scratch-cc06-ag-ki-radiologie/prostate_foundation/WSAttention-Prostate/datatemp/processed/prostate_mask',
 'heatmapdir': '/sc-scratch/sc-scratch-cc06-ag-ki-radiologie/prostate_foundation/WSAttention-Prostate/datatemp/processed/heatmaps/'}

In [8]:
args.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
args.num_classes = 4
args.mil_mode = 'att_trans'
pirads_model = MILModel_3D(
    num_classes=args.num_classes,  
    mil_mode=args.mil_mode 
)
pirads_checkpoint = torch.load(os.path.join(args.project_dir, 'models', 'pirads.pt'), map_location="cpu")
pirads_model.load_state_dict(pirads_checkpoint["state_dict"])
pirads_model.to(args.device)

enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related t

MILModel_3D(
  (attention): Sequential(
    (0): Linear(in_features=512, out_features=2048, bias=True)
    (1): Tanh()
    (2): Linear(in_features=2048, out_features=1, bias=True)
  )
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0-3): 4 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
        )
        (linear1): Linear(in_features=512, out_features=2048, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
        (linear2): Linear(in_features=2048, out_features=512, bias=True)
        (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.0, inplace=False)
        (dropout2): Dropout(p=0.0, inplace=False)
      )
    )
  )
  (myfc): Linear(in_features=512, out_features=4, bias=True)
  (net): ResNet(
    (conv1): Conv3d(3, 64, 

In [31]:
cspca_model = csPCa_Model(backbone=pirads_model).to(args.device)
checkpt = torch.load(os.path.join(args.project_dir, 'models', 'cspca_model.pth'), map_location="cpu")
cspca_model.load_state_dict(checkpt['state_dict'])
cspca_model = cspca_model.to(args.device)

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.


In [33]:
pirads_list = []
pirads_model.eval()
cspca_risk_list = []
cspca_model.eval()
with torch.no_grad():
    for idx, batch_data in enumerate(loader):
        data = batch_data["image"].as_subclass(torch.Tensor).to(args.device)
        logits = pirads_model(data)
        pirads_score= torch.argmax(logits, dim=1)
        pirads_list.append(pirads_score.item())

        output = cspca_model(data)
        output = output.squeeze(1)
        cspca_risk_list.append(output.item())